### Do importu funkcji do symulacji w git codespace

In [ ]:
# Klonowanie repozytorium
!git clone https://github.com/JakDan03/Monte-Carlo-epidemic.git

In [ ]:
# Ewentualne usunięcie repozytorium w celu jego aktualizacji
!rm -rf Monte-Carlo-epidemic

In [ ]:
# instalacja silnika do otwierania Excela
%pip install openpyxl

In [ ]:
import sys
import os

# import funkcji odpowiedzialnych za symulacje
sys.path.append(os.path.abspath("../src"))
from sim_functions import symulacja_choroby_szczepionka

### Analiza wrażliwości - jak rozdysponowanie szczepionek między zdrowych a ozdrowiałych (zmiana parametru alfa) wpłynie na śmiertelność?

In [ ]:
# parametry
import numpy as np

liczba_dni = 1000
liczba_przebiegow = 100
szczepionka_start = 400
alfa_values = np.linspace(0, 1, 11) # od 0 do 1 co 0,1

In [ ]:
# miejsce na średnią liczbę zgonów dla każdej wartości parametru alfa
wyniki = {alfa: 0 for alfa in alfa_values)}

# dla każdej wartości alfy
for alfa_val in alfa_values:
    
    # stwórz miejsce na liczbę zmarłych
    zmarli_func = np.zeros(liczba_przebiegow)

    # dla każdego przebiegu symulacji
    for przebieg in range(liczba_przebiegow):

        # wykonaj symulację
        (   
            zdrowi_k_stan_2,
            zdrowi_m_stan_2,
            urodzeni_k_stan_2,
            urodzeni_m_stan_2,
            zmarli_ogolem_k_stan_2,
            zmarli_ogolem_m_stan_2,
            chorzy_k_stan_2,
            chorzy_m_stan_2,
            ozdrowiali_k_stan_2,
            ozdrowiali_m_stan_2,
            ponownie_chorzy_k_stan_2,
            ponownie_chorzy_m_stan_2,
            zmarli_choroba_k_stan_2,
            zmarli_choroba_m_stan_2,
            zaszczepieni_k_stan,
            zaszczepieni_m_stan
        ) = symulacja_choroby_szczepionka(liczba_dni = liczba_dni, szczepionka_start = szczepionka_start, alfa = alfa_val)

        # zapisz liczbę zmarłych
        zmarli_func[przebieg] = sum(zmarli_ogolem_k_stan_2 + zmarli_ogolem_m_stan_2)

    # zapisz średnią łączną liczbę zmarłych jako wynik
    wyniki[alfa_val] = np.mean(zmarli_func)

In [ ]:
# znajdź wartość alfy minimalizującą liczbę zgonów
alfa_best = alfa_values[np.argmin(wyniki)]
print(f"Najlepsze alfa: {alfa_best}")

### Wizualizacja wyników na wykresie

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("dark_background")

# Wykres
plt.figure(figsize=(8, 5))
plt.plot(list(wyniki.keys()), list(wyniki.values()), marker='o', linestyle='-', color='royalblue')
plt.xlabel("Wartość parametru alfa")
plt.ylabel("Łączna liczba zmarłych")
plt.title("Łączna liczba zmarłych w zależności od strategii szczepień (n=" + str(liczba_przebiegow) + ")")
plt.xticks(alfa_values)
plt.grid(False)
plt.tight_layout()
plt.show()

### Zapis wyników symulacji do csv

In [ ]:
# Tworzenie DataFrame
df_sensitivity = pd.DataFrame({
    'alfa': alfa_values,
    'wynik': wyniki
})

# Zapis do pliku csv
df_sensitivity.to_csv('sensitivity.csv', index=False, encoding='utf-8')